![Header](img/Headers/perform-a-reconciliation.png)

## The Challenge

We are an asset manager who has outsourced our fund accounting. We have multiple clients, each with multiple portfolios containing different investment strategies and priorities.

We are feeling a lot of fee pressure and to maintain profitability we need to keep our costs low.

Outsourcing our fund accounting was a big part of this plan, however there have been a large number of significant discrepancies between our records and those of the fund accountant. Thus we have had to set up an internal 'shadow' team to reconcile the differences between our records and those of the fund accountant. The cost of running this team almost completely negates the cost saving of outsourcing our fund accounting. In addition, there have been many times were we have just had to accept the records of the fund accountant as being correct.

This has lead our portfolio managers to become incredibly frustrated as they can't accurately determine their position and are being forced into trading on the basis of unreliable information.

We desperately need a solution that allows us to improve our reconciliation process so that we can easily identify & reconcile the discrepancies between our records and those of the fund accountant.

## The Solution

LUSID has been designed to help address challenges such as this.

Using LUSID we can automate the reconciliation processs to help our investment managers get a better understanding of their position and remove the need to have a separate shadow accounting team.

We can do this by:

1. Creating one environment for our internal records and one for those of the fund accountant
2. Creating the current holdings for our clients in both environments
3. Completing a day of trading and updating our internal records with the day's trades
4. Receiving a daily holdings report from our fund accountant and updating the fund accountant record
5. Reconciling our internal records with those of the fund accountant
6. Building a system to automatically identify and flag any discrepancies

At the end of this process our LUSID environment will look like the below:

![Scopes](img/transparency-matchedexceptions.gif)

# 0) Import Libraries and Initialise LUSID client

In [1]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import reconciliation as rc_tools

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('LUSID API Version :', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version : 0.5.3299.0


# 1) Create our client portfolios in an internal and fund accountant environment

### a) Create our Scopes

In [2]:
# Fetch our scopes
internal_scope_code = rc_tools.fetch_scope("internal-records", 'Internal Scope Code')
fund_accountant_scope_code = rc_tools.fetch_scope("fund-accountant-records", 'Fund Accountant Scope Code')

scopes = [internal_scope_code, fund_accountant_scope_code]

Internal Scope Code: internal-records-37c3-68ee-ab13-ca
Fund Accountant Scope Code: fund-accountant-records-37c3-68ee-ab17-6c


### b) Create our portfolios 

#### Import our portfolio names and groups

In [3]:
client_portfolios = import_data.fetch_portfolio_names('portfolios.csv')
prettyprint.portfolios(client_portfolios)

Portfolio group: client-A-portfolios
   Portfolio: client-A-strategy-balanced
   Portfolio: client-A-strategy-tech
   Portfolio: client-A-strategy-growth


Portfolio group: client-B-portfolios
   Portfolio: client-B-strategy-balanced
   Portfolio: client-B-strategy-energy
   Portfolio: client-B-strategy-fixedincome
   Portfolio: client-B-strategy-international
   Portfolio: client-B-strategy-usgovt


Portfolio group: client-C-portfolios
   Portfolio: client-C-strategy-balanced
   Portfolio: client-C-strategy-fixedincome




#### Create the portfolios and their groups in LUSID in both internal and fund accountant scopes

In [4]:
# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=895)).isoformat()

# Iterate over our two scopes, this will create our portfolios and portfolio groups in both scopes
for scope in scopes:
    
    # Iterate over our portfolio groups selecting the name of the group and the list of portfolios
    for portfolio_group_code, portfolio_group in client_portfolios.items():

        # Loop over our list of portfolios selecting the portfolio code
        for portfolio_code in portfolio_group:

            # Create our portfolio
            rc_tools.create_portfolio(
                scope=scope,
                display_name=portfolio_code,
                code=portfolio_code,
                base_currency='GBP',
                description=portfolio_code,
                created=portfolio_creation_date)
            
        # Create our portfolio group and add the portfolio to it
        rc_tools.create_portfolio_group(scope, portfolio_group, portfolio_group_code)
        
print ('All portfolios have been created')

Portfolio Created
Scope: internal-records-37c3-68ee-ab13-ca
Code: client-A-strategy-balanced
Portfolio Effective From: 2017-04-15 15:25:16.745528+00:00
Portfolio Created On: 2019-09-27 15:25:17.401024+00:00

Portfolio Created
Scope: internal-records-37c3-68ee-ab13-ca
Code: client-A-strategy-tech
Portfolio Effective From: 2017-04-15 15:25:16.745528+00:00
Portfolio Created On: 2019-09-27 15:25:17.884258+00:00

Portfolio Created
Scope: internal-records-37c3-68ee-ab13-ca
Code: client-A-strategy-growth
Portfolio Effective From: 2017-04-15 15:25:16.745528+00:00
Portfolio Created On: 2019-09-27 15:25:18.168069+00:00

Portfolio Group Created
Name: client-A-portfolios
Scope: internal-records-37c3-68ee-ab13-ca
Code: client-A-portfolios
Portfolios Inside Group: 
client-A-strategy-growth
client-A-strategy-tech
client-A-strategy-balanced


Portfolio Created
Scope: internal-records-37c3-68ee-ab13-ca
Code: client-B-strategy-balanced
Portfolio Effective From: 2017-04-15 15:25:16.745528+00:00
Portfolio

# 2) Create our current holdings for our clients 

### a) Create our instrument universe

#### Import our instrument universe

In [5]:
instrument_universe = import_data.fetch_instrument_universe('instruments.csv')
prettyprint.instruments(instrument_universe, 'ClientInternal')

Amazon_Nasdaq_AMZN: ClientInternal - imd_34634534
Apple_Nasdaq_AAPL: ClientInternal - imd_35345345
BP_LondonStockEx_BP: ClientInternal - imd_43535553
BurfordCapital_LondonStockEx_BUR: ClientInternal - imd_43534356
EKFDiagnostics_LondonStockEx_EKF: ClientInternal - imd_34535355
Glencore_LondonStockEx_GLEN: ClientInternal - imd_34534555
JustEat_LondonStockEx_JE: ClientInternal - imd_35436366
Kingfisher_LondonStockEx_KGF: ClientInternal - imd_34535552
MicroFocus_LondonStockEx_MCRO: ClientInternal - imd_34567338
RELXGroup_LondonStockEx_REL: ClientInternal - imd_43532542
Sage_LondonStockEx_SGE: ClientInternal - imd_23536673
TESCO_LondonStockEx_TSCO: ClientInternal - imd_34634673
UKGiltTreasury_2.0_2025: ClientInternal - imd_34534536
UKGiltTreasury_3.5_2045: ClientInternal - imd_54234532
UKGiltTreasury_3.75_2021: ClientInternal - imd_34643653
UKGiltTreasury_4.5_2034: ClientInternal - imd_34534534
USTreasury_2.00_2021: ClientInternal - imd_34535347
USTreasury_6.875_2025: ClientInternal - imd_

#### Load the instruments into LUSID

In [6]:
# Upsert our instruments into LUSID
instrument_response = rc_tools.batch_upsert_instruments(instrument_universe)

prettyprint.instrument_response(instrument_response)

Instruments Successfully Upserted: 


,Instrument,ClientInternal ID,LUSID Instrument ID
0,Amazon_Nasdaq_AMZN,imd_34634534,LUID_YN4RBVWK
1,Whitebread_LondonStockEx_WTB,imd_35349900,LUID_QWZAHTER
2,UKGiltTreasury_4.5_2034,imd_34534534,LUID_LNW8I9LO
3,Apple_Nasdaq_AAPL,imd_35345345,LUID_ZKHI8KWW
4,USTreasury_2.00_2021,imd_34535347,LUID_YUM2HZSQ
5,UKGiltTreasury_3.75_2021,imd_34643653,LUID_06KBEF9L
6,UKGiltTreasury_3.5_2045,imd_54234532,LUID_HN4OK91W
7,Sage_LondonStockEx_SGE,imd_23536673,LUID_EWBBCKNB
8,RELXGroup_LondonStockEx_REL,imd_43532542,LUID_194V2WFU
9,Glencore_LondonStockEx_GLEN,imd_34534555,LUID_DJUKBNV2


#### Add LUSID Instrument Ids (LUIDs) and Cash Instruments

In [7]:
# Add our LUIDs as new identifier to each instrument so that we can use it in our calls later
rc_tools.add_luids(instrument_response, instrument_universe)
    
# We will also add some cash instruments to our universe
rc_tools.add_cash_instrument(instrument_universe, "GBP")
rc_tools.add_cash_instrument(instrument_universe, "USD")
            
prettyprint.instruments(instrument_universe, 'LUID')

Amazon_Nasdaq_AMZN: LUID - LUID_YN4RBVWK
Apple_Nasdaq_AAPL: LUID - LUID_ZKHI8KWW
BP_LondonStockEx_BP: LUID - LUID_0BVQT88M
BurfordCapital_LondonStockEx_BUR: LUID - LUID_EYEDZCIV
EKFDiagnostics_LondonStockEx_EKF: LUID - LUID_I9AW9SJE
Glencore_LondonStockEx_GLEN: LUID - LUID_DJUKBNV2
JustEat_LondonStockEx_JE: LUID - LUID_MA53URVP
Kingfisher_LondonStockEx_KGF: LUID - LUID_H6LL1EIM
MicroFocus_LondonStockEx_MCRO: LUID - LUID_6M4P66NL
RELXGroup_LondonStockEx_REL: LUID - LUID_194V2WFU
Sage_LondonStockEx_SGE: LUID - LUID_EWBBCKNB
TESCO_LondonStockEx_TSCO: LUID - LUID_TWDX74LL
UKGiltTreasury_2.0_2025: LUID - LUID_ZTMR6A5H
UKGiltTreasury_3.5_2045: LUID - LUID_HN4OK91W
UKGiltTreasury_3.75_2021: LUID - LUID_06KBEF9L
UKGiltTreasury_4.5_2034: LUID - LUID_LNW8I9LO
USTreasury_2.00_2021: LUID - LUID_YUM2HZSQ
USTreasury_6.875_2025: LUID - LUID_U65YYD85
WPP_LondonStockEx_WPP: LUID - LUID_DAJ31M6Y
Whitebread_LondonStockEx_WTB: LUID - LUID_QWZAHTER
GBP_Cash: LUID - GBP
USD_Cash: LUID - USD


### b) Set our take on balances

#### Import initial holdings

In [8]:
client_holdings = import_data.fetch_client_take_on_balances('holdings.csv')
prettyprint.holdings(client_holdings)

Portfolio: client-A-strategy-balanced

Instrument Name: WPP_LondonStockEx_WPP
Quantity: 2956000
CostPrice: 8.71

Instrument Name: UKGiltTreasury_2.0_2025
Quantity: 375856
CostPrice: 8.71

Instrument Name: JustEat_LondonStockEx_JE
Quantity: 4026354
CostPrice: 5.4639999999999995

Instrument Name: UKGiltTreasury_3.75_2021
Quantity: 486913
CostPrice: 108.126

Instrument Name: GBP_Cash
Quantity: 3000000
CostPrice: 1.0




Portfolio: client-A-strategy-tech

Instrument Name: MicroFocus_LondonStockEx_MCRO
Quantity: 687994
CostPrice: 14.535

Instrument Name: Sage_LondonStockEx_SGE
Quantity: 2599653
CostPrice: 5.77

Instrument Name: GBP_Cash
Quantity: 784000
CostPrice: 1.0




Portfolio: client-A-strategy-growth

Instrument Name: BurfordCapital_LondonStockEx_BUR
Quantity: 853486
CostPrice: 14.06

Instrument Name: EKFDiagnostics_LondonStockEx_EKF
Quantity: 925925
CostPrice: 0.27

Instrument Name: GBP_Cash
Quantity: 150000
CostPrice: 1.0




Portfolio: client-B-strategy-balanced

Instrument Name: 

#### Set these Initial Holdings in LUSID

In [9]:
holdings_effective_date = (datetime.now(pytz.UTC) - timedelta(days=2)).isoformat()

# Iterate over our portfolios
for portfolio_name, portfolio in client_holdings.items():
    
    # Create our adjust holdings request for the holding in each portfolio
    holding_adjustments = rc_tools.adjust_holdings_request(portfolio, instrument_universe)

    # Iterate over our two scopes
    for scope in scopes:

        # Set the holdings for our portfolios
        rc_tools.set_holdings_response(scope, portfolio_name, holdings_effective_date, holding_adjustments)

print ('All portfolios have been set')

Holdings Successfully Set for Portfolio
Scope: internal-records-37c3-68ee-ab13-ca
Code: client-A-strategy-balanced
Holdings Effective From: 2019-09-25 15:25:27.760251+00:00
Holdings Created On: 2019-09-27 15:25:28.252283+00:00

Holdings Successfully Set for Portfolio
Scope: fund-accountant-records-37c3-68ee-ab17-6c
Code: client-A-strategy-balanced
Holdings Effective From: 2019-09-25 15:25:27.760251+00:00
Holdings Created On: 2019-09-27 15:25:28.737224+00:00

Holdings Successfully Set for Portfolio
Scope: internal-records-37c3-68ee-ab13-ca
Code: client-A-strategy-tech
Holdings Effective From: 2019-09-25 15:25:27.760251+00:00
Holdings Created On: 2019-09-27 15:25:29.386342+00:00

Holdings Successfully Set for Portfolio
Scope: fund-accountant-records-37c3-68ee-ab17-6c
Code: client-A-strategy-tech
Holdings Effective From: 2019-09-25 15:25:27.760251+00:00
Holdings Created On: 2019-09-27 15:25:29.978979+00:00

Holdings Successfully Set for Portfolio
Scope: internal-records-37c3-68ee-ab13-ca


# 3) Complete a day of trading and update our internal records

### a) Import Yesterdays Transactions

In [10]:
client_transactions, yesterday_trade_open = import_data.client_transactions('transactions.csv', instrument_universe)
prettyprint.transactions(client_transactions)

Portfolio: client-A-strategy-balanced

Transaction: tid_35b77104-7c72-4515-af2b-d22bb45e6edd
Instrument Name: WPP_LondonStockEx_WPP
Instrument LUID: LUID_DAJ31M6Y
Type: Sell
Units: 265600
Price: 8.91
Currency: GBP
Transaction Date: 2019-09-26T09:00:00+00:00
Settlement Date: 2019-09-27T08:00:00+00:00



Portfolio: client-A-strategy-tech

Transaction: tid_2ac903c9-3615-4716-80d7-1608d40617dc
Instrument Name: MicroFocus_LondonStockEx_MCRO
Instrument LUID: LUID_6M4P66NL
Type: Buy
Units: 15074
Price: 13.2867
Currency: GBP
Transaction Date: 2019-09-26T13:00:00+00:00
Settlement Date: 2019-10-01T08:00:00+00:00



Portfolio: client-B-strategy-balanced

Transaction: tid_4f192c6a-e74b-464e-95c2-62a18b092bd8
Instrument Name: Kingfisher_LondonStockEx_KGF
Instrument LUID: LUID_H6LL1EIM
Type: Sell
Units: 325000
Price: 2.345
Currency: GBP
Transaction Date: 2019-09-26T11:00:00+00:00
Settlement Date: 2019-09-29T08:00:00+00:00

Transaction: tid_6fc1a588-f19b-4222-93d5-c35dcf4c7d46
Instrument Name: UKGilt

### b) Upsert our transactions to LUSID for each of our internal portfolios

In [11]:
# Iterate over our portfolios
for portfolio_name, portfolio in client_transactions.items():

    # Initialise a list to hold our transactions for each portfolio
    batch_transaction_requests = []

    # Iterate over the transactions for each portfolio
    for transaction_id, transaction in portfolio.items():

        # Add transaction requests
        rc_tools.add_transaction_requests(batch_transaction_requests, transaction_id, transaction)

    # Upsert transactions to LUSID
    rc_tools.add_transaction_response(internal_scope_code, portfolio_name, batch_transaction_requests)

Transactions Successfully Upserted into Portfolio
Scope: internal-records-37c3-68ee-ab13-ca
Code: client-A-strategy-balanced
Transactions Effective From: 2019-09-26 09:00:00+00:00
Transactions Created On: 2019-09-27 15:25:39.522822+00:00

Transactions Successfully Upserted into Portfolio
Scope: internal-records-37c3-68ee-ab13-ca
Code: client-A-strategy-tech
Transactions Effective From: 2019-09-26 13:00:00+00:00
Transactions Created On: 2019-09-27 15:25:40.094725+00:00

Transactions Successfully Upserted into Portfolio
Scope: internal-records-37c3-68ee-ab13-ca
Code: client-B-strategy-balanced
Transactions Effective From: 2019-09-26 16:12:00+00:00
Transactions Created On: 2019-09-27 15:25:40.881527+00:00

Transactions Successfully Upserted into Portfolio
Scope: internal-records-37c3-68ee-ab13-ca
Code: client-B-strategy-fixedincome
Transactions Effective From: 2019-09-26 12:00:00+00:00
Transactions Created On: 2019-09-27 15:25:41.450075+00:00

Transactions Successfully Upserted into Portf

# 4) Receive our daily holdings report and update the fund accountant record

### a) Import our fund accountant's records

In [12]:
fund_accountant_daily_holdings_report = import_data.fetch_fund_accountant_daily_holdings_report('fundaccountantreport.csv')
prettyprint.holdings(fund_accountant_daily_holdings_report)

Portfolio: client-A-strategy-balanced

Instrument Name: WPP_LondonStockEx_WPP
Quantity: 2690400.0
CostPrice: 8.71

Instrument Name: UKGiltTreasury_2.0_2025
Quantity: 375856.0
CostPrice: 8.71

Instrument Name: JustEat_LondonStockEx_JE
Quantity: 4026354.0
CostPrice: 5.4639999999999995

Instrument Name: UKGiltTreasury_3.75_2021
Quantity: 486913.0
CostPrice: 108.126

Instrument Name: GBP_Cash
Quantity: 5366496.0
CostPrice: 1.0




Portfolio: client-A-strategy-tech

Instrument Name: MicroFocus_LondonStockEx_MCRO
Quantity: 703068.0
CostPrice: 14.5082

Instrument Name: Sage_LondonStockEx_SGE
Quantity: 2599653.0
CostPrice: 5.77

Instrument Name: GBP_Cash
Quantity: 583716.2842
CostPrice: 1.0




Portfolio: client-A-strategy-growth

Instrument Name: BurfordCapital_LondonStockEx_BUR
Quantity: 853486.0
CostPrice: 14.06

Instrument Name: EKFDiagnostics_LondonStockEx_EKF
Quantity: 925925.0
CostPrice: 0.27

Instrument Name: GBP_Cash
Quantity: 150000.0
CostPrice: 1.0




Portfolio: client-B-strategy-b

###  b) Adjust the holdings for portfolios in the fund accountant's scope according to their report

In [13]:
today = datetime.now(pytz.UTC)
t = time(hour=6, minute=30)
this_morning = pytz.utc.localize(datetime.combine(today, t)).isoformat()

# Iterate over our portfolios
for portfolio_name, portfolio in fund_accountant_daily_holdings_report.items():
    
    # Create our adjust holdings request for the holding in each portfolio
    holding_adjustments = rc_tools.adjust_holdings_request(portfolio, instrument_universe)

    # Adjust the holdings only in our fund accountant's scope
    rc_tools.adjust_holdings_response(fund_accountant_scope_code, portfolio_name, this_morning, holding_adjustments)

print ('All holdings adjusted')

Holdings Successfully Adjusted for Portfolio
Scope: fund-accountant-records-37c3-68ee-ab17-6c
Code: client-A-strategy-balanced
Adjusted Holdings Effective From: 2019-09-27 06:30:00+00:00
Adjusted Holdings Created On: 2019-09-27 15:25:42.036943+00:00

Holdings Successfully Adjusted for Portfolio
Scope: fund-accountant-records-37c3-68ee-ab17-6c
Code: client-A-strategy-tech
Adjusted Holdings Effective From: 2019-09-27 06:30:00+00:00
Adjusted Holdings Created On: 2019-09-27 15:25:42.548107+00:00

Holdings Successfully Adjusted for Portfolio
Scope: fund-accountant-records-37c3-68ee-ab17-6c
Code: client-A-strategy-growth
Adjusted Holdings Effective From: 2019-09-27 06:30:00+00:00
Adjusted Holdings Created On: 2019-09-27 15:25:43.164358+00:00

Holdings Successfully Adjusted for Portfolio
Scope: fund-accountant-records-37c3-68ee-ab17-6c
Code: client-B-strategy-balanced
Adjusted Holdings Effective From: 2019-09-27 06:30:00+00:00
Adjusted Holdings Created On: 2019-09-27 15:25:43.768094+00:00

Ho

# 5) Reconcile our internal records with those of the fund accountant

### Compare our internal and fund accountant's portfolios to find discrepancies

In [14]:
# Initialise our reconiled portfolios to dictionary to hold any reconciliation breaks
reconciled_portfolios = {}

# Iterate over our portfolios
for portfolio_name in fund_accountant_daily_holdings_report:

    # Define our internal and fund accountant portfolios
    internal_portfolio = rc_tools.define_portfolio(internal_scope_code, portfolio_name, this_morning)
    fund_accountant_portfolio = rc_tools.define_portfolio(fund_accountant_scope_code, portfolio_name, this_morning)
    
    # Reconcile our portfolios and list any breaks
    rc_tools.reconciliation(internal_portfolio, fund_accountant_portfolio, portfolio_name, reconciled_portfolios)
            
prettyprint.reconciliation(reconciled_portfolios)

Portfolio: client-B-strategy-balanced

Reconciliation Break
Instrument: LUID_LNW8I9LO
Internal Units: 87982.0
Fund Accountant Units: 77481.0
Difference In Units: -10501.0
Internal Cost: 12367805.7
Fund Accountant Cost: 10891659.13
Difference In Cost: -1476146.57
Currency: GBP


Reconciliation Break
Instrument: CCY_GBP
Internal Units: 237978.4280000001
Fund Accountant Units: 1714125.0
Difference In Units: 1476146.572
Internal Cost: 237978.4280000001
Fund Accountant Cost: 1714125.0
Difference In Cost: 1476146.572
Currency: GBP


Portfolio: client-C-strategy-balanced

Reconciliation Break
Instrument: LUID_TWDX74LL
Internal Units: 1864441.0
Fund Accountant Units: 2206441.0
Difference In Units: 342000.0
Internal Cost: 3675745.43
Fund Accountant Cost: 4349998.43
Difference In Cost: 674253.0
Currency: GBP


Reconciliation Break
Instrument: CCY_GBP
Internal Units: 17937283.0
Fund Accountant Units: 17292100.0
Difference In Units: -645183.0
Internal Cost: 17937283.0
Fund Accountant Cost: 1729210

# 6) Build a system to automatically identify and flag discrepancies

### a) Compare our internal and fund accountant's portfolios at the close of trade yesterday

In [15]:
trade_close_time = (yesterday_trade_open + timedelta(hours=8)).isoformat()

# Initialise our reconiled portfolios to dictionary to hold any reconciliation breaks
reconciled_portfolios_trade_close = {}

# Iterate over our portfolios
for portfolio_name in fund_accountant_daily_holdings_report:

    # Define our internal and fund accountant portfolios
    internal_portfolio = rc_tools.define_portfolio(internal_scope_code, portfolio_name, trade_close_time)
    fund_accountant_portfolio = rc_tools.define_portfolio(fund_accountant_scope_code, portfolio_name, this_morning)
    
    # Reconcile our portfolios and list any breaks
    rc_tools.reconciliation(internal_portfolio, fund_accountant_portfolio, portfolio_name, reconciled_portfolios_trade_close)
            
prettyprint.reconciliation(reconciled_portfolios_trade_close, True)

No reconciliation breaks


### b) Identify transactions that happened after the close of trading in all of our portfolios

In [16]:
late_trades = rc_tools.get_late_trades(
    report=fund_accountant_daily_holdings_report, 
    scope=internal_scope_code, 
    from_transaction_date=trade_close_time, 
    to_transaction_date=this_morning)

prettyprint.trades(late_trades)

Portfolio: client-B-strategy-balanced

Transaction: tid_6fc1a588-f19b-4222-93d5-c35dcf4c7d46
Instrument: LUID_LNW8I9LO
Type: Buy
Units: 10501.0
Price: 140.572
Currency: GBP
Transaction Date: 2019-09-26 16:12:00+00:00


Portfolio: client-C-strategy-balanced

Transaction: tid_adefc478-aca8-4a9b-89b1-57f49a31d1b5
Instrument: LUID_TWDX74LL
Type: Sell
Units: 342000.0
Price: 1.8865
Currency: GBP
Transaction Date: 2019-09-26 16:18:00+00:00




### c) Check whether these late trades match our reconciliation breaks

In [17]:
# Initialise our dictionary to hold our exceptions
matched_exceptions = []
  
# Iterate over our portfolios and the late trades we have identified
for portfolio_name, late_trade in late_trades.items():
    # Loop over each late trade
    for trade in late_trade.values:
        # Check if it matches a reconciliation break
        for reconciliation_break in reconciled_portfolios[portfolio_name].values:
            '''
            Here we use the absolute difference in units to reduce the complexity in identifying if a
            transaction is a buy or a sell and whether we need to match a positive or negative difference in
            units. In practice we can add in the ability to check the direction as well
            '''
            units = abs(reconciliation_break.difference_units)
            instrument_uid = reconciliation_break.instrument_uid

            # If the instrument id and units match, we have identified the cause of the reconciliation break
            if (trade.instrument_uid == instrument_uid) and (trade.units == units):
                matched_exceptions.append((trade, reconciliation_break, portfolio_name))

prettyprint.exceptions(matched_exceptions)

Transaction: tid_6fc1a588-f19b-4222-93d5-c35dcf4c7d46
Instrument: LUID_LNW8I9LO
Type: Buy
Units: 10501.0
Price: 140.572
Currency: GBP

MATCHES

Reconciliation Break
Instrument: LUID_LNW8I9LO
Internal Units: 87982.0
Fund Accountant Units: 77481.0
Difference In Units: -10501.0
Internal Cost: 12367805.7
Fund Accountant Cost: 10891659.13
Difference In Cost: -1476146.57
Currency: GBP



Transaction: tid_adefc478-aca8-4a9b-89b1-57f49a31d1b5
Instrument: LUID_TWDX74LL
Type: Sell
Units: 342000.0
Price: 1.8865
Currency: GBP

MATCHES

Reconciliation Break
Instrument: LUID_TWDX74LL
Internal Units: 1864441.0
Fund Accountant Units: 2206441.0
Difference In Units: 342000.0
Internal Cost: 3675745.43
Fund Accountant Cost: 4349998.43
Difference In Cost: 674253.0
Currency: GBP





### d) Flag these late trades with a LUSID property
#### With this we can confirm whether they are in tomorrow's report from the fund accountant

In [18]:
# Create our property for late trades
property = models.CreatePropertyDefinitionRequest(
    domain='Transaction',
    scope=internal_scope_code,
    code='late_trade',
    value_required=True,
    display_name='late_trade',
    data_type_id=models.ResourceId(
        scope='system',
        code='boolean'))

client.property_definitions.create_property_definition(
    definition=property)

# Flag our matched trades with the late trade property
for exception in matched_exceptions:
    rc_tools.add_property(exception, property, "True")

Property Successfully Added On Transaction in Portfolio
Transaction: tid_6fc1a588-f19b-4222-93d5-c35dcf4c7d46
Scope: internal-records-37c3-68ee-ab13-ca
Code: client-B-strategy-balanced
Property Effective From: 2019-09-26 16:12:00+00:00
Property Created On: 2019-09-27 15:25:53.742838+00:00

Property Successfully Added On Transaction in Portfolio
Transaction: tid_adefc478-aca8-4a9b-89b1-57f49a31d1b5
Scope: internal-records-37c3-68ee-ab13-ca
Code: client-C-strategy-balanced
Property Effective From: 2019-09-26 16:18:00+00:00
Property Created On: 2019-09-27 15:25:54.311375+00:00

